In [30]:
import pandas as pd
import numpy as np
import re
import torch

In [31]:
def CleanText(text):
    text = re.sub(r'''[\[|\]]''', "", text).split()
    text = np.array(text, dtype="float64")
    return text

In [32]:
df = pd.read_csv("domain_data_with_identities.csv")
df['comment_text'] = df['comment_text'].apply(lambda text: CleanText(text))

In [33]:
# Updating values for training_data
training_data = df[df['split'] == 'train']

# Getting test_data
test_data = df[df['split'] == 'test']

# Getting validation_data
validation_data = df[df['split'] == 'val']

In [34]:
X_train = np.array(training_data['comment_text'].values.tolist())

X_test = np.array(test_data['comment_text'].values.tolist())

X_val = np.array(validation_data['comment_text'].values.tolist())

In [35]:
samples, features = X_train.shape

In [ ]:
from diffprivlib.models import PCA

# The only permitted `svd_solver` is 'full'.  Specifying the ``svd_solver`` option will result in a warning

# If n_components is not set all components are kept

#Centered: If False, the data will be centered before calculating the principal components

#If ``centered=False``, half of epsilon is used to calculate the
#        differentially private mean to center the data prior to the calculation of principal components


#data_norm: The max l2 norm of any row of the data.  This defines the spread of data that will be protected by
#        differential privacy.
#Matrix norms: the Frobenius norm: #https://www.datacamp.com/community/tutorials/tutorial-machine-learning-basics-norms

data_norm = np.linalg.norm(X_train)


#model = PCA(n_components = features, centered='False', epsilon=0.1, data_norm = data_norm)
model = PCA(epsilon=0.1)
model.fit(X_train)

In [37]:
PCAModel001 = torch.load('PCAeps0,01.pth')

In [38]:
transformedXtrain = PCAModel001.transform(X_train)

In [39]:
for i, ind in enumerate(training_data.index):
    df.at[ind, 'comment_text'] = transformedXtrain[i]

In [40]:
df.to_csv(r'C:\Users\frede\Desktop\domain_data_with_identities_private_xtrain0,01.csv', index=False, header=True)